In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = dt.date(2022, 6, 1)
itp_id = 159

In [3]:
pbar = tqdm()

0it [00:00, ?it/s]

In [4]:
rt_day = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet


In [5]:
rt_day.export_views_gcs()

In [6]:
importlib.reload(rt)
importlib.reload(shared_utils)

<module 'shared_utils' from '/home/jovyan/data-analyses/_shared_utils/shared_utils/__init__.py'>

In [143]:
rt.OperatorDayAnalysis._generate_stop_delay_view(rt_day)

could not generate delays for trip t_1057276_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1057308_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1439236_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1057273_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1547204_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1057313_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1057370_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1437743_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1437746_b_25985_tn_0
strptime() argument 1 must be str, not None
could not generate delays for trip t_1547202_b_25985_tn_0
strptime() argument 1 mu

In [145]:
df = rt_day.debug_dict['delays'] >> filter(_.trip_id == 't_1057276_b_25985_tn_0') >> arrange(_.stop_sequence)

In [146]:
def fix_arrival_time(gtfs_timestring: str) -> tuple[str, int]:
    """Reformats a GTFS timestamp (which allows the hour to exceed
    24 to mark service day continuity)
    to standard 24-hour time.
    """
    extra_day = 0
    if not gtfs_timestring: # preserve none if time not provided
        return None, extra_day
    split = gtfs_timestring.split(":")
    hour = int(split[0])

    if hour >= 24:
        extra_day = 1
        split[0] = str(hour - 24)
        corrected = (":").join(split)
        return corrected.strip(), extra_day

    else:
        return gtfs_timestring.strip(), extra_day

In [147]:
df.arrival_time = df.arrival_time.map(lambda x: fix_arrival_time(x)[0])

In [148]:
df >> head(3)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id
7,783995,Main & Third,POINT (-252059.032 118071.269),p_4684,0.134045,-8876945536004734190,t_1057276_b_25985_tn_0,1,15:30:00,2042,4,1
9,784033,Main & D St.,POINT (-252082.645 117466.756),p_4684,604.994549,-8876945536004734190,t_1057276_b_25985_tn_0,2,15:33:00,2042,4,1
1,783997,Bowling Alley,POINT (-252624.574 117092.033),p_4684,3911.131886,-8876945536004734190,t_1057276_b_25985_tn_0,3,None,2042,4,1


In [149]:
import numpy as np

In [150]:
df['arrival_time'] = df.apply(lambda x:
                                    dt.datetime.combine(analysis_date,
                                                        dt.datetime.strptime(x.arrival_time, '%H:%M:%S').time()) if x.arrival_time else np.nan,
                                                        axis = 1) ## format scheduled arrival times

In [151]:
df.arrival_time.iloc[0]

Timestamp('2022-06-01 15:30:00')

In [152]:
## TODO timezone convert back and forth to enable interpolation, review other time handling code

In [158]:
def interpolate_arrival_times(df):
    '''
    Interpolate between provided arrival_times linearly with distance
    '''
    interp_df = df.dropna(subset=['arrival_time'])
    yp = interp_df.arrival_time.to_numpy()
    yp = yp.astype('datetime64[s]').astype('float64')
    xp = interp_df.shape_meters.to_numpy()

    interpolator = lambda x: np.interp(x, xp, yp)
    df.arrival_time = df.apply(lambda x: interpolator(x.shape_meters) if pd.isnull(x.arrival_time) else x.arrival_time,
                               axis = 1)
    df.arrival_time = df.arrival_time.to_numpy().astype('datetime64[s]')
    return df

In [159]:
interpolate_arrival_times(df)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id
7,783995,Main & Third,POINT (-252059.032 118071.269),p_4684,0.134045,-8876945536004734190,t_1057276_b_25985_tn_0,1,2022-06-01 15:30:00,2042,4,1
9,784033,Main & D St.,POINT (-252082.645 117466.756),p_4684,604.994549,-8876945536004734190,t_1057276_b_25985_tn_0,2,2022-06-01 15:33:00,2042,4,1
1,783997,Bowling Alley,POINT (-252624.574 117092.033),p_4684,3911.131886,-8876945536004734190,t_1057276_b_25985_tn_0,3,2022-06-01 15:41:51,2042,4,1
3,784032,Social Security & DMV,POINT (-253004.033 117101.848),p_4684,2227.969017,-8876945536004734190,t_1057276_b_25985_tn_0,4,2022-06-01 15:39:19,2042,4,1
4,784031,Parallel Dr. & Lakeport Blvd.,POINT (-252970.009 116911.875),p_4684,2015.223509,-8876945536004734190,t_1057276_b_25985_tn_0,5,2022-06-01 15:39:00,2042,4,1
17,784030,Bruno's (between driveways),POINT (-252412.486 117067.151),p_4684,1299.447072,-8876945536004734190,t_1057276_b_25985_tn_0,6,2022-06-01 15:35:57,2042,4,1
8,784029,Grocery Outlet,POINT (-252142.200 117010.238),p_4684,4516.515981,-8876945536004734190,t_1057276_b_25985_tn_0,7,2022-06-01 15:42:46,2042,4,1
15,784028,Hospice Thrift Store,POINT (-252181.314 116593.900),p_4684,4934.032066,-8876945536004734190,t_1057276_b_25985_tn_0,8,2022-06-01 15:43:23,2042,4,1
2,784027,S Main & Peckham K-mart,POINT (-252195.365 116193.470),p_4684,5334.834153,-8876945536004734190,t_1057276_b_25985_tn_0,9,2022-06-01 15:44:00,2042,4,1
13,784026,By Rotten Robbies,POINT (-251935.398 115304.243),p_4684,6264.761484,-8876945536004734190,t_1057276_b_25985_tn_0,10,2022-06-01 15:45:16,2042,4,1


In [138]:
test.arrival_time.to_numpy().astype('datetime64[s]')

array(['2022-06-01T15:30:00', '2022-06-01T15:33:00',
       '2022-06-01T15:41:51', '2022-06-01T15:39:19',
       '2022-06-01T15:39:00', '2022-06-01T15:35:57',
       '2022-06-01T15:42:46', '2022-06-01T15:43:23',
       '2022-06-01T15:44:00', '2022-06-01T15:45:16',
       '2022-06-01T15:51:34', '2022-06-01T15:56:33',
       '2022-06-01T15:57:00', '2022-06-01T15:57:37',
       '2022-06-01T15:58:00', '2022-06-01T16:15:00',
       '2022-06-01T16:25:00', '2022-06-01T16:26:05',
       '2022-06-01T16:30:00'], dtype='datetime64[s]')

In [32]:
rt_day.trs

,stop_id,stop_name,geometry,shape_id,shape_meters
0,783958,Lake Transit Yard,POINT (-226224.908 102987.490),p_4684,40447.291914
1,783958,Lake Transit Yard,POINT (-226224.908 102987.490),p_4685,37043.157781
2,783958,Lake Transit Yard,POINT (-226224.908 102987.490),p_4677,71181.490055
3,783958,Lake Transit Yard,POINT (-226224.908 102987.490),p_4676,56717.229418
4,784231,Buckingham Country Club,POINT (-238062.185 112878.168),p_4679,9833.815106
...,...,...,...,...,...
648,784155,Lakeshore Boulevard and Park Way,POINT (-251513.420 121684.904),p_1275848,18141.237056
649,784258,Admiral Rd.,POINT (-236584.832 100295.263),p_4682,10798.486096
650,784258,Admiral Rd.,POINT (-236584.832 100295.263),p_1275833,7786.943824
651,784121,Turn out across from Chevron,POINT (-252127.395 131343.955),p_4704,50430.748954


In [48]:
df.arrival_time.iloc[0].timestamp()

1654097400.0

In [49]:
# set system time
os.environ["TZ"] = "America/Los_Angeles"
time.tzset()

In [50]:
dt.datetime.fromtimestamp(1654097400.0)

datetime.datetime(2022, 6, 1, 8, 30)

In [39]:
df.arrival_time.interpolate(method='time')

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got time

In [ ]:
df